In [1]:
%pip install rdflib>=6.3

zsh:1: 6.3 not found
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path
import json
import re
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, OWL


EX = Namespace("http://example.org/exercise#")
SULO = Namespace("https://w3id.org/sulo/")

ATTR_CLASSES = {
    "ImpactLevel": EX.ImpactLevelQuality,
    "Intensity": EX.IntensityQuality,
    "BalanceRequirement": EX.BalanceRequirementQuality,
    "JointInvolvement": EX.JointInvolvementQuality,
}

def slug(text: str) -> str:
    """Safe URI suffix."""
    return re.sub(r'[^A-Za-z0-9]+', '_', text.strip()).strip('_')

def ensure(g: Graph, uri: URIRef, rdf_type: URIRef, label: str | None = None):
    """Ensure a URI exists in the graph with given type and label."""
    if (uri, RDF.type, None) not in g:
        g.add((uri, RDF.type, rdf_type))
    if label:
        g.add((uri, RDFS.label, Literal(label, lang='en')))

In [ ]:


def inject_exercises(g, exercise_rows):
    """
    Add exercise information to the graph.
    `exercise_rows` is a list of dicts loaded from JSON files.
    """

    ensure(g, EX.ExerciseProcess, OWL.Class)
    g.add((EX.ExerciseProcess, RDFS.subClassOf, SULO.Process))
    

    ensure(g, EX.ExerciseEquipment, OWL.Class)
    
    exercise_types = ["Strength", "Aerobic", "Anaerobic", "Stretching"]
    for ex_type in exercise_types:
        type_class = EX[f"{ex_type}Exercise"]
        ensure(g, type_class, OWL.Class, f"{ex_type} Exercise")
        g.add((type_class, RDFS.subClassOf, EX.ExerciseProcess))
    
    for row in exercise_rows:
        exercise_name = row["ExerciseName"].strip()
        exercise_type = row["ExerciseType"].strip()
        description = row.get("Description", "")
        
        exercise_uri = EX[f"{slug(exercise_name)}Exercise"]
        type_class = EX[f"{exercise_type}Exercise"]
        
        ensure(g, exercise_uri, type_class, exercise_name)
        
        if description:
            g.add((exercise_uri, RDFS.comment, Literal(description, lang='en')))

        attr_set_uri = EX[f"AttributeSet_{slug(exercise_name)}"]
        ensure(g, attr_set_uri, SULO.Set, f"attributes for {exercise_name}")
        
        g.add((exercise_uri, SULO.refersTo, attr_set_uri))
        

        for key, q_class in ATTR_CLASSES.items():
            if key == "JointInvolvement":
                continue
                
            val = row.get(key, None)
            
            if val and val.upper() != "NULL":

                attr_uri = EX[f"{slug(val)}{key}"]
                ensure(g, attr_uri, SULO.InformationObject, f"{val.lower()} {key.lower()}")
                g.add((attr_uri, SULO.hasValue, Literal(val.lower(), datatype=XSD.string)))
                g.add((attr_uri, SULO.refersTo, q_class))
                g.add((attr_set_uri, SULO.refersTo, attr_uri))
        
        joint_vals = row.get("JointInvolvement", [])
        if isinstance(joint_vals, str) and joint_vals.upper() != "NULL":
            joint_vals = [joint_vals]
            
        if joint_vals and joint_vals[0].lower() != "none":
            for joint in joint_vals:
                v_uri = EX[f"{slug(joint)}InvV"]
                ensure(g, v_uri, SULO.InformationObject, f"{joint} involvement")
                g.add((v_uri, SULO.hasValue, Literal(joint, datatype=XSD.string)))
                g.add((v_uri, SULO.refersTo, EX.JointInvolvementQuality))
                g.add((attr_set_uri, SULO.refersTo, v_uri))
        
        equipment_vals = row.get("Equipment", [])
        if isinstance(equipment_vals, str) and equipment_vals.upper() != "NULL":
            equipment_vals = [equipment_vals]
            
        if equipment_vals and equipment_vals[0].lower() != "none":
            for equip in equipment_vals:
                if equip.lower() == "none":
                    continue
                

                equip_uri = EX[f"{slug(equip)}Equipment"]
                ensure(g, equip_uri, EX.ExerciseEquipment, equip)
                
                equip_info_uri = EX[f"{slug(equip)}EquipmentInfo"]
                ensure(g, equip_info_uri, SULO.InformationObject, f"{equip} equipment")
                g.add((equip_info_uri, SULO.hasValue, Literal(equip, datatype=XSD.string)))
                g.add((equip_info_uri, SULO.refersTo, equip_uri))
                
                g.add((attr_set_uri, SULO.refersTo, equip_info_uri))

def load_json_files(*json_paths):
    """Load and combine data from multiple JSON files."""
    all_rows = []
    for path in json_paths:
        with open(path, 'r') as f:
            rows = json.load(f)
            all_rows.extend(rows)
    return all_rows

def inject_exercises_from_files(schema_path, out_path, *exercise_json_paths):
    """
    Load schema, inject exercises, and save the result.
    """
    g = Graph()
    g.parse(schema_path)
    
    exercise_rows = load_json_files(*exercise_json_paths)
    inject_exercises(g, exercise_rows)
    
    g.serialize(out_path, format="turtle")
    print(f"Saved output to {out_path}")



In [ ]:
if __name__ == "__main__":
    schema = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/sulo_with_guidelines.ttl')
    out = Path('sulo_with_exercises.ttl')

    exercise_jsons = [
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Exercises/aerobic_exercises.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Exercises/anaerobic_exercises.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Exercises/strength_exercises.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Exercises/stretching_exercises.json'),
    ]

    inject_exercises_from_files(schema, out, *exercise_jsons)

Saved output to sulo_with_exercises.ttl
